In [ ]:
# importing Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.color_palette("husl", 4)
import plotly.graph_objs as go
import plotly.express as px
import plotly.offline as py
import plotly.offline as py
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# importing all three dataframes
districts = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')
products = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')

In [ ]:
# combine all engaement  dataframes who having full engagement in 2020 some data will lost
from tqdm import tqdm
PATH = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
temp = []


for district in tqdm(districts.district_id.unique()):
    df = pd.read_csv(f'{PATH}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    if df.time.nunique() == 366:
        temp.append(df)
            
big_df = pd.concat(temp)
engagement = big_df.reset_index(drop=True)

In [ ]:
# Only consider districts and products with full engagement data
districts = districts[districts.district_id.isin(engagement.district_id.unique())].reset_index(drop=True)
products = products[products['LP ID'].isin(engagement.lp_id.unique())].reset_index(drop=True)

In [ ]:
#first we will analyse products
display(products.head(),products.info(),products.describe())

In [ ]:
# making missing plot
def missing_plot(df):
    l1 = []
    l2 = []
    for i in df.columns:
        l1.append(i)
        a = (df[i].isna().sum())/df[i].count()*100
        l2.append(a)
    l1 = pd.DataFrame(l1)
    l2 = pd.DataFrame(l2)
    df2 = pd.concat([l1,l2],axis=1)
    df2.columns = ['Names','Percentage']
    plt.figure(figsize=(13,6))
    g = sns.barplot(x = 'Names',y = 'Percentage',data = df2)
    for index, row in df2.iterrows():
        g.text(row.name,row.Percentage, round(row.Percentage,2), color='black', ha="center")

In [ ]:
missing_plot(products)
# removing missing values on Primary Essential function
products = products[products['Primary Essential Function'].notna()].reset_index(drop = True)

In [ ]:
# ploting  unique values of every feature:
def unique(df):
    l1 = [i for i in df.columns]
    l2 = [df[i].nunique() for i in df.columns]
    l1 = pd.DataFrame(l1)
    l2 = pd.DataFrame(l2)
    df2 = pd.concat([l1,l2],axis=1)
    df2.columns = ['Names','Unique_values']
    plt.figure(figsize=(15,7))
    g = sns.barplot(x = 'Names',
            y = 'Unique_values',
            data = df2)
    for index, row in df2.iterrows():
        g.text(row.name,row.Unique_values,round(row.Unique_values,2),color='black', ha="center")

In [ ]:
# checking unique values
unique(products)

In [ ]:
def label_enco(df):
    temp_df = df.copy()
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for i in temp_df.columns:
        if temp_df[i].dtypes=='object':
            temp_df[i] = le.fit_transform(temp_df[i])
    return temp_df
products2 = label_enco(products)

In [ ]:
fix,axes = plt.subplots(4,1,figsize=(15,15))
sns.boxplot(data = products2,x ='Product Name',ax = axes[0])
axes[0].set_title('Product Name')

sns.boxplot(data = products2,x ='Provider/Company Name',ax=axes[1])
axes[1].set_title('Provider/Company Name')

sns.boxplot(data = products2,x ='Sector(s)',ax = axes[2])
axes[2].set_title('Sector(s)')

sns.boxplot(data = products2,x ='Primary Essential Function',ax = axes[3])
axes[3].set_title('Primary Essential Function')

plt.tight_layout()
plt.show()

In [ ]:
# seaborn heatmap
sns.heatmap(label_enco(products).corr(),annot=True,linecolor='white',linewidths=0.5)

In [ ]:
# There are 372 unique Products out of 372 Products so we can skip this ind var
products['Product Name'].shape

In [ ]:
def donutchart(df,col):
    trace = go.Pie(labels = df[col].unique(),values=df[col].value_counts(),textfont=dict(size=18),opacity = 0.5,marker=dict(colorssrc = 'tealrose',line=dict(color = '#000000',width = 1.5)),hole = 0.6)
    layout = dict(title = f'Donut chart of {col}')
    fig = dict(data = [trace],layout = layout)
    py.iplot(fig)

In [ ]:
# frequency with "Provider/Company Name" 
freq = products.groupby(['Provider/Company Name']).count()
freq.sort_values(by=['Product Name'], ascending=False )

In [ ]:
donutchart(freq,'Product Name')

In [ ]:
freq = products.groupby(['Sector(s)','Provider/Company Name']).count()
freq.sort_values(by=['Product Name'], ascending=False )

In [ ]:
donutchart(freq,'Product Name')

In [ ]:
# find which companies are focusing which sectors:
freq = products[products['Sector(s)'] == 'PreK-12']
freq = freq.groupby(['Provider/Company Name']).count()
freq.sort_values(by=['Product Name'],ascending=False)[:5]

In [ ]:
donutchart(freq,'Product Name')

In [ ]:
freq= products[products['Sector(s)'] == 'PreK-12; Higher Ed; Corporate']
freq = freq.groupby(['Provider/Company Name']).count()
freq.sort_values(by=['Product Name'], ascending=False )[:5]

In [ ]:
donutchart(freq,'Product Name')

In [ ]:
freq = products[products['Sector(s)'] == 'Corporate']
freq = freq.groupby(['Provider/Company Name']).count()
freq.sort_values(by=['Product Name'], ascending=False )

In [ ]:
donutchart(freq,'Product Name')

In [ ]:
freq = products[products['Sector(s)']=='Higher Ed; Corporate']
freq = freq.groupby(['Provider/Company Name']).count()
freq.sort_values(by=['Product Name'],ascending=False)

In [ ]:
donutchart(freq,'Product Name')

In [ ]:
# which company is focusing on which 'Primary Essential Function
products['Primary Essential Function'].unique()
freq = products.groupby(['Provider/Company Name','Primary Essential Function']).count()
freq.sort_values(by=['Product Name'],ascending = False)[:10]

In [ ]:
donutchart(freq,'Product Name')

In [ ]:
import plotly.express as px
df = px.data.tips()
sunb = products[['Primary Essential Function','Provider/Company Name','Sector(s)']].dropna()
sunb = sunb.groupby(['Primary Essential Function','Provider/Company Name']).size().reset_index(name = 'count')
fig = px.sunburst(sunb,path = ['Primary Essential Function','Provider/Company Name'], values='count')
fig.show()

In [ ]:
df =  products[['Product Name','Primary Essential Function','Provider/Company Name','Sector(s)']]
df = df.dropna()
df = df.groupby(['Provider/Company Name','Product Name','Primary Essential Function','Sector(s)']).size().reset_index(name = 'count')
df.sort_values(by=['count'],ascending = False)

In [ ]:
# now we we will explore district data

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
districts['state_abbrev'] = districts['state'].replace(us_state_abbrev)
districts_info = districts['state_abbrev'].value_counts().to_frame().reset_index(drop=False)
districts_info.columns = ['state_abbrev','num_districts']
fig = go.Figure()
layout = dict(title = "Number of availible school in states",geo_scope = 'usa')
fig.add_trace(go.Choropleth(locations = districts_info.state_abbrev,
                           zmax = 1,
                           z = districts_info.num_districts,
                           locationmode = 'USA-states',
                           marker_line_color = 'black',
                           geo = 'geo',
                           colorscale=px.colors.sequential.Hot)
             )
fig.update_layout(layout)
fig.show()
districts = districts.drop(['state_abbrev'],axis = 1)
display()

In [ ]:
display(districts.head(),districts.info(),districts.describe())

In [ ]:
# Checking missing values
missing_plot(districts)
# remvoing missing values on basis states most imp feature of this data frame
districts = districts[districts.state.notna()].reset_index(drop = True)

In [ ]:
# unique values in districts columns
unique(districts)

In [ ]:
# column 'county_connections_ratio' only contains one unique value, i guess it doesn't contain that much useful information
districts = districts.drop(['county_connections_ratio'],axis = 1)

In [ ]:
fix,axes = plt.subplots(3,1,figsize=(15,10))
sns.countplot(data = districts,x ='pct_black/hispanic',palette = 'GnBu',ax = axes[0])
axes[0].set_title('pct_black/hispanic')
sns.countplot(data = districts,x ='pct_free/reduced',palette = 'GnBu',ax=axes[1])
axes[1].set_title('pct_free/reduced')
sns.countplot(data = districts,x ='pp_total_raw',palette = 'GnBu',ax = axes[2])
axes[2].set_title('pp_total_raw')
plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
sunb = districts[['state','locale']].dropna()
sunb = sunb.groupby(['state','locale']).size().reset_index(name = 'count')
fig = px.sunburst(sunb,path = ['state','locale'], values='count')
fig.show()

In [ ]:
def bub1(b):
    a = b.value_counts()
    fig = px.scatter(a,y =a.values,size=a.values,color=a.index,hover_name=a.index,size_max=60,
                     title='State with number of schools')
    fig.show()

In [ ]:
bub1(districts['state'])

In [ ]:
bub1(districts['locale'])

In [ ]:
# now we will deal with pct_black/hispanic pct_free/reduced pp_total_raw these columns we will convert [0, 0.2[ them into mean of their value
def paralta(v):
    v = str(v)
    if v[0] == '[':
        v = (v[1:len(v)-1]).replace(" ", "")
        v1,v2 = float(v.split(',')[0]),float(v.split(',')[1])
        return round((v1+v2)/2,3)
    else:
        pass

In [ ]:
# applying function 
districts['pct_black/hispanic'] = districts['pct_black/hispanic'].apply(paralta)
districts['pct_free/reduced'] = districts['pct_free/reduced'].apply(paralta)
districts['pp_total_raw'] = districts['pp_total_raw'].apply(paralta)

In [ ]:
# Filling values if there is,
print(districts['pct_black/hispanic'].isna().sum()) # No NaN values

print(districts['pct_free/reduced'].isna().sum())# 21 NaN values, fll them Mean values
districts['pct_free/reduced'].fillna(value = districts['pct_free/reduced'].mean(),inplace = True)

print(districts['pp_total_raw'].isna().sum())# 49 NaN values, fill them Mean values
districts['pp_total_raw'].fillna(value = districts['pp_total_raw'].mean(),inplace = True)

In [ ]:
# checking outliners 
fix,axes = plt.subplots(3,1,figsize=(15,15))
sns.boxplot(data = districts,x ='pct_black/hispanic',ax = axes[0])
axes[0].set_title('pct_black/hispanic')
sns.boxplot(data = districts,x ='pct_free/reduced',ax=axes[1])
axes[1].set_title('pct_free/reduced')
sns.boxplot(data = districts,x ='pp_total_raw',ax = axes[2])
axes[2].set_title('pp_total_raw')
plt.tight_layout()
plt.show()

In [ ]:
# this function deals with outliners by filling them median value
def outliners(df,i):
    q1 = df[i].quantile(0.25)
    q3 = df[i].quantile(0.75)
    iqr = q3-q1
    low = q1-1.5*iqr
    high = q3+1.5*iqr
    out1 = df[(df[i]<low)].values
    out2 = df[(df[i]>high)].values
    mean = df[i].mean()
    df[i].replace(out1,mean,inplace=True)
    df[i].replace(out2,mean,inplace=True)
    return df

In [ ]:
districts['pct_black/hispanic'] = outliners(districts,'pct_black/hispanic')
districts['pp_total_raw'] = outliners(districts,'pp_total_raw')

In [ ]:
# visualising states ,locale in context with 'pct_black/hispanic' (treating them as object)
from plotly.subplots import make_subplots

label1 = [i for i in districts['state'].unique()]
label2 = [i for i in districts['locale'].unique()]

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['State', 'locale'])
fig.add_trace(go.Pie(labels=label1, values=districts['pct_black/hispanic'], scalegroup='one',
                     name="state"), 1, 1)
fig.add_trace(go.Pie(labels=label2, values=districts['pct_black/hispanic'], scalegroup='one',
                     name="locale"), 1, 2)

fig.update_layout(title_text='states and locale in context with pct_black/hispanic')
fig.show()


# visualising states locale in context with 'pct_free/reduced' (treating them as object)
from plotly.subplots import make_subplots

label1 = [i for i in districts['state'].unique()]
label2 = [i for i in districts['locale'].unique()]

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['State', 'locale'])
fig.add_trace(go.Pie(labels=label1, values=districts['pct_free/reduced'], scalegroup='one',
                     name="state"), 1, 1)
fig.add_trace(go.Pie(labels=label2, values=districts['pct_free/reduced'], scalegroup='one',
                     name="locale"), 1, 2)

fig.update_layout(title_text='states and locale in context with pct_free/reduced')
fig.show()


# visualising states locale in context with 'pp_total_raw' (treating them as object)
from plotly.subplots import make_subplots

label1 = [i for i in districts['state'].unique()]
label2 = [i for i in districts['locale'].unique()]

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['State', 'locale'])
fig.add_trace(go.Pie(labels=label1, values=districts['pp_total_raw'], scalegroup='one',
                     name="state"), 1, 1)
fig.add_trace(go.Pie(labels=label2, values=districts['pp_total_raw'], scalegroup='one',
                     name="locale"), 1, 2)

fig.update_layout(title_text='states and locale in context with pp_total_raw')
fig.show()

In [ ]:
# checking outliners 
fix,axes = plt.subplots(3,1,figsize=(15,15))
sns.boxplot(data = districts,x ='pct_black/hispanic',ax = axes[0])
axes[0].set_title('pct_black/hispanic')
sns.boxplot(data = districts,x ='pct_free/reduced',ax=axes[1])
axes[1].set_title('pct_free/reduced')
sns.boxplot(data = districts,x ='pp_total_raw',ax = axes[2])
axes[2].set_title('pp_total_raw')
plt.tight_layout()
plt.show()

In [ ]:
# treating 'pct_black/hispanic''pct_free/reduced','pp_total_raw' as a numeric value 
# so we can which state has highest 'pct_black/hispanic''pct_free/reduced','pp_total_raw'
temp1 = districts['pct_black/hispanic'].apply(lambda x: float(x))
temp2 = districts['pct_free/reduced'].apply(lambda x: float(x))
temp3 = districts['pp_total_raw'].apply(lambda x: float(x))


sns.displot(districts, x=temp1, hue='state', kind="kde", fill=True,ax = axes[0])
sns.displot(districts, x=temp2, hue='state', kind="kde", fill=True,ax = axes[1])
sns.displot(districts, x=temp3, hue='state', kind="kde", fill=True,ax = axes[2])
plt.tight_layout()
plt.show()

In [ ]:
# treating 'pct_black/hispanic''pct_free/reduced','pp_total_raw' as a numeric value 
# so we can see which locale has highest 'pct_black/hispanic''pct_free/reduced','pp_total_raw'

temp1 = districts['pct_black/hispanic'].apply(lambda x: float(x))
temp2 = districts['pct_free/reduced'].apply(lambda x: float(x))
temp3 = districts['pp_total_raw'].apply(lambda x: float(x))



sns.displot(districts, x=temp1, hue='locale', kind="kde", fill=True,ax = axes[0])
sns.displot(districts, x=temp2, hue='locale', kind="kde", fill=True,ax = axes[1])
sns.displot(districts, x=temp3, hue='locale', kind="kde", fill=True,ax = axes[2])
plt.tight_layout()
plt.show()

In [ ]:
# now we will work on engagement DATA

In [ ]:
display(engagement.head(),engagement.info(),engagement.isna().sum())

In [ ]:
# converting date and time datatype from object to Datetime datatype
engagement['time'] = pd.to_datetime(engagement['time'])

In [ ]:
# there are some NAN in  lp id and a lot of NAN in engagement index
# i guess  droping them is a safe play
engagement.dropna(inplace = True)

In [ ]:
# we will Feature Engineer 'time' into month
month = engagement['time'].dt.month
month = pd.DataFrame(month)
month.columns = ['month']

# we will Feature Engineer 'time' into month
day = engagement['time'].dt.day
day = pd.DataFrame(day)
day.columns = ['day']

engagement = pd.concat([engagement,month,day],axis = 1)
engagement.head()

In [ ]:
# typcasting district_id from int to string so we can visualise district id in y axis 
engagement['district_id'] = engagement['district_id'].apply(lambda x: str(x))

In [ ]:
# now we will see which district has 'engagement_index' in different months
# from jan to december
for i in range(1,13):
    temp = engagement[engagement['month']==i]
    temp = temp.sort_values(by=['engagement_index'],ascending = False)[:20]
    sns.displot(temp, x=temp['engagement_index'], hue=temp['district_id'], kind="kde")
    plt.xlabel("District id")
    plt.ylabel("Values")
    plt.title(f"Highest Engagement index in {i} month") # You can comment this line out if you don't need title
plt.show()

In [ ]:
# now we will see which district has higher 'pct access' in different months
# from jan to december
for i in range(1,13):
    temp = engagement[engagement['month']==i]
    temp = temp.sort_values(by=['pct_access'],ascending = False)[:20]
    sns.displot(temp, x=temp['pct_access'], hue=temp['district_id'], kind="kde")
    plt.xlabel("District id")
    plt.ylabel("Values")
    plt.title(f"Highest pct access in {i} month") # You can comment this line out if you don't need title
    plt.show()

In [ ]:
# this function will give  the lp_id with highest  pct_access or engagement_index on daily basis
def func(df,feat,m,v):
    y = []
    lab  = []
    def no_days(m):
        import calendar
        import datetime
        a = list(calendar.monthrange(2021,m))[1]
        return a
    for j in range(1,no_days(m)+1):
        temp = df[df['month']==m]
        temp = temp[temp['day']==j]
        a = temp.sort_values(feat,ascending=False)[:1]
        y.append(float(a[feat][:1]))
        lab.append(int(a[v][:1]))
    x = [i for i in range(1,len(y)+1)]
    lab = [str(i) for i in lab]
    return x,y,lab
# we will run above function in loop from range 1 to 12 
# so we will get the results for 12 months

def func2(df,feat,v):
    # this function will plot bar plot
    def barplott(x,y,lab,m,feat):
        fig = px.bar(x, y =y,text = lab)
        fig.update_layout(title = dict(text = f'{m} month - {feat}'),)
        fig.show()
        
    for i in range(1,13):
        x,y,lab = func(df,feat,i,v)
        barplott(x,y,lab,i,feat)


In [ ]:
# plotting for 'pct_access'
func2(engagement,'pct_access','lp_id')

In [ ]:
# plotting for 'engagement_index'
func2(engagement,'engagement_index','lp_id')

In [ ]:
# now we will see what is the prcnt  of lp_id that have higher number of 'pct_access' and 'engagement_index' with help of pie chart
def func3(df,feat,v):
    def piechart(l1,feat,m):
        def pcnt(c,tc):
            return (c/tc)*100
        l2 = []
        for i in l1:
            l2.append(pcnt(l1.count(i),len(l1)))
        df2 = pd.DataFrame(list(zip(l1, l2)),columns =['names', 'values'])
        fig = px.pie(df2, values='values', names='names')
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.update_layout(title = dict(text = f'{m} month - {feat}'),)
        fig.show()
    for i in range(1,13):
        _,_,l1 = func(df,feat,i,v)
        piechart(l1,feat,i)

In [ ]:
func3(engagement,'pct_access','lp_id')

In [ ]:
func3(engagement,'engagement_index','lp_id')

In [ ]:
# now we will analyse top  'lp id' of every month in context with pct_access / engagement_index
# with help of products and engagement csv's

In [ ]:
# this function will give top 'lp id's of individual day  in context with pct_access/enagement_index
# then we will append them into a list and remove repeated lp id 
def func_5(df,feat,v,df2,feat2):
    L = []
    for m in range(1,13):
        _,_,l = func(df,feat,m,v)
        [L.append(i) for i in l]
    # this list contain all the unique lp_ids  which are popular all year in context with pct access
    L = list(set([int(i) for i in L]))
    #  now we will create a dataframe where we have only top  id in L 
    top = (df2[df2[feat2].isin(L)]).reset_index()
    display(top)
    ls = list([i for i in top.columns])
  # so we can easily get the idea what kind of products are mostly used by the lp_id easily 
  # with help of sunburst plot
    import plotly.express as px
    df = px.data.tips()
    sunb = top[ls].dropna()
    sunb = sunb.groupby(ls).size().reset_index(name = 'count')
    fig = px.sunburst(sunb,path = ls
                      , values='count')
    fig.show()

In [ ]:
# top used products by top lp id in context with pct_access
products.drop('URL',axis = 'columns',inplace = True)
func_5(engagement,'pct_access','lp_id',products,'LP ID') 

In [ ]:
# top used products by top lp id in context with engagement_index
func_5(engagement,'engagement_index','lp_id',products,'LP ID') 

In [ ]:
# end of EDA